In [198]:
%load_ext autoreload
%autoreload 2
from TNR import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [199]:
chi={}
chi['w'],chi['y'],chi['u'],chi['v']=6,6,6,6
chiw,chiy,chiu,chiv=chi['w'],chi['y'],chi['u'],chi['v']

# J=1
# htemp=(KP(s0,sx,sx,s0)+KP(s0,sz,sz,s0))+\
#     J*(KP(sx,sx,s0,s0)+KP(sz,sz,s0,s0)+KP(s0,s0,sx,sx)+KP(s0,s0,sz,sz))/2
# htemp=KP(sz,sz)+J*(KP(sx,s0)+KP(s0,sx))/2
# htemp=(KP(sz,s0)+KP(s0,sz))/2+(KP(sx,s0)+KP(s0,sx))/2
# htemp=KP(sx,sx)+KP(sz,sz)
# htemp=-KP(R,R.conj())-J*(KP(M,np.eye(3))+KP(np.eye(3),M))/2
# htemp=-KP(R,R.conj())-KP(M,M)
# htemp+=htemp.T.conj()

# d=int(np.sqrt(htemp.shape[0]))
# Ainit=expm(-0.002*htemp).reshape(d,d,d,d)
# A0=Asplit(Ainit,chi)
# for i in range(7):
#     A0=coarsen(A0,chi)

# classical ising:
na=np.newaxis
s=np.array([1,-1])
Tc=2/np.log(1+np.sqrt(2))
T=1*Tc
A0=np.exp(-1/T*(s[:,na,na,na]*s[na,:,na,na]+s[na,:,na,na]*s[na,na,:,na]\
                +s[na,na,:,na]*s[na,na,na,:]+s[na,na,na,:]*s[:,na,na,na]))
A0/=np.linalg.norm(A0)

In [224]:
RGsteps=10
A_list=[None]*(RGsteps+1)
Anorm_list=[None]*(RGsteps+1)
B_list,U_list,vl_list,vr_list,yl_list,yr_list,D_list,w_list,u_list,v_list=[[None]*RGsteps for i in range(10)]
A_list[0]=A0

for i in range(RGsteps):
    print('RG step: ',i)
    Aold=A_list[i]
    B,U,vl,vr,yl,yr,D,w,u,v,A,Anorm=doTNR(Aold,chi)
    B_list[i],U_list[i],vl_list[i],vr_list[i],yl_list[i],yr_list[i],D_list[i],w_list[i],u_list[i],v_list[i],A_list[i+1],Anorm_list[i+1]=\
    B,U,vl,vr,yl,yr,D,w,u,v,A,Anorm
    print('A.shape= ',A.shape)
    print()

RG step:  0
iter=0, U.Uenv=0.58
iter=100, U.Uenv=0.58
error in U, vl, vr optimization: 5.12577e-16
error in yl, yr, D optimization: 7.27387e-15
error in w optimization: 9.03172e-16
iter=0, u.uenv=0.463478, v.venv=0.441666
iter=1000, u.uenv=0.915927, v.venv=0.915927
iter=2000, u.uenv=0.915927, v.venv=0.915927
Adiff 0.0922636
A.shape=  (4, 4, 4, 4)

RG step:  1
iter=0, U.Uenv=0.492758
iter=100, U.Uenv=0.492796
iter=200, U.Uenv=0.492806
iter=300, U.Uenv=0.492809
iter=400, U.Uenv=0.49281
iter=500, U.Uenv=0.49281
iter=600, U.Uenv=0.49281
iter=700, U.Uenv=0.49281
iter=800, U.Uenv=0.49281
iter=900, U.Uenv=0.49281
iter=1000, U.Uenv=0.49281
error in U, vl, vr optimization: 0.00048738
error in yl, yr, D optimization: 0.0127057
error in w optimization: 0.0278938
iter=0, u.uenv=0.440844, v.venv=0.426816
iter=1000, u.uenv=0.979327, v.venv=0.979327
iter=2000, u.uenv=0.979327, v.venv=0.979327
Adiff 0.120366
A.shape=  (6, 6, 6, 6)

RG step:  2
iter=0, U.Uenv=0.517359
iter=100, U.Uenv=0.517444
iter=200

In [105]:
i=7
A=A_list[i+1]

In [235]:
T=np.einsum(A,[2,1,2,0])
# ew,_=np.linalg.eigh(T)
ew = scipy.sparse.linalg.eigsh(T, k=100, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[2]-el[0]))[:100]

c:\programdata\miniconda3\lib\site-packages\scipy\sparse\linalg\eigen\arpack\arpack.py:1254: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  RuntimeWarning)


array([-0.        ,  0.1185881 ,  1.        ,  1.03526507,  1.11012191,
        1.42195287])

In [237]:
T=A[0,:,0,:]
ew,_=np.linalg.eigh(T)
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[2]-el[0]))[:100]

array([-0.        ,  0.53734485,  1.        ,  0.91949011,  0.6253713 ,
        0.57335888])

In [238]:
T2=np.einsum(A,[4,2,5,0],A,[5,3,4,1]).reshape(chiw**2,chiw**2)
ew,_=np.linalg.eigh(T2)
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[2]-el[0]))[:100]

array([-0.        ,  0.12498022,  1.        ,  1.12335006,  1.12421145,
        1.98774168,  2.00162317,  2.02697953,  2.04074836,  2.14480648,
        2.14828669,  2.17457332,  2.70948425,  2.7492312 ,  2.7497438 ,
        2.88486088,  3.0478844 ,  3.21521455,  3.30954823,  3.31570005,
        3.35564174,  3.44284917,  3.79468674,  4.03144508,  4.08477725,
        4.09811851,  4.10259439,  4.55638605,  4.56196266,  4.42458208,
        4.22736378,  4.11736856,  3.884238  ,  3.5024575 ,  3.31580867,
        3.27952689])

In [239]:
T3=np.einsum(A,[10,3,8,0],A,[8,4,9,1],A,[9,5,10,2]).reshape(chiw**3,chiw**3)
ew = scipy.sparse.linalg.eigsh(T3, k=100, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[2]-el[0]))[:100]

array([-0.        ,  0.12557751,  1.        ,  1.12417724,  1.12417724,
        1.99928523,  1.99928523,  2.00783879,  2.00783879,  2.13305496,
        2.13305496,  2.13391197,  2.99484746,  3.02226469,  3.04008761,
        3.04008761,  3.04803807,  3.11777385,  3.12159258,  3.157412  ,
        3.16859833,  3.16859833,  3.21063772,  3.90421335,  3.91345114,
        3.91345114,  3.94182719,  3.94182719,  4.04721177,  4.04721177,
        4.05701377,  4.1283819 ,  4.1283819 ,  4.1734247 ,  4.1734247 ,
        4.18636669,  4.18636669,  4.30269029,  4.36442643,  4.45024191,
        4.45024191,  4.46927825,  4.46927825,  4.62702991,  4.843758  ,
        4.843758  ,  4.97509996,  4.98832185,  4.98832185,  5.13513025,
        5.13513025,  5.22219337,  5.24624473,  5.24624473,  5.34194669,
        5.34194669,  5.34585414,  5.36877996,  5.36877996,  5.40721601,
        5.40956881,  5.53567869,  5.53567869,  5.65783339,  5.65783339,
        5.67619994,  5.74340877,  5.74340877,  5.78810005,  5.78

In [240]:
T4=np.einsum(A,[11,4,8,0],A,[8,5,9,1],A,[9,6,10,2],A,[10,7,11,3]).reshape(chiw**4,chiw**4)
ew,_ = np.linalg.eig(T4)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[2]-el[0]))[:100]

array([-0.        ,  0.12591164,  1.        ,  1.12397447,  1.12397447,
        1.99983768,  1.99997458,  2.00306896,  2.00306896,  2.12643182,
        2.12905483,  2.12944401,  3.00453135,  3.00453135,  3.01254739,
        3.01569266,  3.01691209,  3.12568275,  3.12568275,  3.1394174 ,
        3.1394174 ,  3.14532227,  3.14532227,  3.99490718,  4.00612003,
        4.01344961,  4.02162664,  4.04391256,  4.04391256,  4.04505071,
        4.05574914,  4.05574914,  4.11939986,  4.12362097,  4.1444107 ,
        4.14449174,  4.16025206,  4.16318961,  4.17766789,  4.18094592,
        4.26441083,  4.96637003,  4.96637003,  4.98545589,  5.01687769,
        5.01959283,  5.03656792,  5.03656792,  5.05780751,  5.05780751,
        5.06368734,  5.10555148,  5.10555148,  5.12579794,  5.13420632,
        5.14528313,  5.14528313,  5.16011322,  5.16011322,  5.18305417,
        5.18305417,  5.1918607 ,  5.1918607 ,  5.2562973 ,  5.2562973 ,
        5.35169484,  5.45592533,  5.52237149,  5.52237149,  5.53

In [242]:
T4=np.einsum(A,[11,4,8,0],A,[8,5,9,1],A,[9,6,10,2],A,[10,7,11,3]).reshape(chiw**4,chiw**4)
ew = scipy.sparse.linalg.eigsh(T4, k=100, which='LM', return_eigenvectors=False)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[2]-el[0]))[:100]

array([-0.        ,  0.12591164,  1.        ,  1.12397447,  1.12397447,
        1.99983768,  1.99997458,  2.00306896,  2.00306896,  2.12643182,
        2.12905483,  2.12944401,  3.00453135,  3.00453135,  3.01254739,
        3.01569266,  3.01691209,  3.12568275,  3.12568275,  3.1394174 ,
        3.1394174 ,  3.14532227,  3.14532227,  3.99490718,  4.00612003,
        4.01344961,  4.02162664,  4.04391256,  4.04391256,  4.04505071,
        4.05574914,  4.05574914,  4.11939986,  4.12362097,  4.1444107 ,
        4.14449174,  4.16025206,  4.16318961,  4.17766789,  4.18094592,
        4.26441083,  4.96637003,  4.96637003,  4.98545589,  5.01687769,
        5.01959283,  5.03656792,  5.03656792,  5.05780751,  5.05780751,
        5.06368734,  5.10555148,  5.10555148,  5.12579794,  5.13420632,
        5.14528313,  5.14528313,  5.16011322,  5.16011322,  5.18305417,
        5.18305417,  5.1918607 ,  5.1918607 ,  5.2562973 ,  5.2562973 ,
        5.35169484,  5.45592533,  5.52237149,  5.52237149,  5.53

In [12]:
T4reduced=np.einsum(A,[11,0,8,0],A,[8,5,9,1],A,[9,6,10,2],A,[10,3,11,3]).reshape(chiw**2,chiw**2)
ew = scipy.sparse.linalg.eigsh(T4reduced, k=100, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[9]-el[0]))[:100]

c:\programdata\miniconda3\lib\site-packages\scipy\sparse\linalg\eigen\arpack\arpack.py:1254: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  RuntimeWarning)


array([-0.00000000e+00,  4.99596599e-09,  9.20580353e-01,  9.23027805e-01,
        9.74246459e-01,  9.91745214e-01,  9.96888023e-01,  9.97677177e-01,
        9.98760396e-01,  1.00000000e+00,  1.03712251e+00,  1.04902022e+00,
        1.07013854e+00,  1.08046930e+00,  1.12311929e+00,  1.17195892e+00,
        1.18890350e+00,  1.24217920e+00,  1.24406100e+00,  1.27621656e+00,
        1.27699451e+00,  1.28690860e+00,  1.28208019e+00,  1.27945905e+00,
        1.22885842e+00,  1.21879832e+00,  1.19262162e+00,  1.10913150e+00,
        1.10420462e+00,  1.08104846e+00,  1.06700641e+00,  1.05314342e+00,
        9.94678778e-01,  9.41076610e-01,  4.05513452e-01,  4.05513447e-01])

In [35]:
#eval mag
tensor_list={}
tensor_list['A'],tensor_list['U'],tensor_list['vl'],tensor_list['vr'],tensor_list['yl'],tensor_list['yr'],tensor_list['D'],tensor_list['w'],tensor_list['u'],tensor_list['v']=A_list,U_list,vl_list,vr_list,yl_list,yr_list,D_list,w_list,u_list,v_list
op=sz

A0,Az=Asplit(Ainit,chi,op=op)
for i in range(8):
    A0,Az=coarsen(A0,chi,Az)
MOq=MOq_(A0,Az)
eval_op(MOq,tensor_list)  

vertical truncation err: 0.33200000533597757
horizontal truncation err: 1.1928251353259178e-05
Aout shape: (6, 6, 6, 6)


ValueError: shapes (2,2) and (3,3,6) not aligned: 2 (dim 1) != 3 (dim 1)

In [70]:
# classical ising:
tensor_list={}
tensor_list['A'],tensor_list['U'],tensor_list['vl'],tensor_list['vr'],tensor_list['yl'],tensor_list['yr'],tensor_list['D'],tensor_list['w'],tensor_list['u'],tensor_list['v']=A_list,U_list,vl_list,vr_list,yl_list,yr_list,D_list,w_list,u_list,v_list
op=sz

A0=A_list[0]
MO=MO_(A0,op)
eval_op(MO,tensor_list)

RGsteps: 0 , <O> = 0j
RGsteps: 1 , <O> = 0j
RGsteps: 2 , <O> = 0j
RGsteps: 3 , <O> = (1.4645132209427232e-08+0j)
RGsteps: 4 , <O> = (-1.0092542614917613e-09+0j)
RGsteps: 5 , <O> = (1.6494869855828184e-08+0j)
RGsteps: 6 , <O> = (1.810992269371651e-08+0j)
RGsteps: 7 , <O> = (-3.9155420681136e-08+0j)
RGsteps: 8 , <O> = (-1.81152375193032e-07+0j)
RGsteps: 9 , <O> = (-8.091429269816573e-07+0j)
RGsteps: 10 , <O> = (-3.4081639479815316e-06+0j)
RGsteps: 11 , <O> = (-1.3772852410857965e-05+0j)
RGsteps: 12 , <O> = (-5.5243157568508106e-05+0j)
RGsteps: 13 , <O> = (-0.0002211243697913897+0j)
RGsteps: 14 , <O> = (-0.0008846486804189272+0j)
RGsteps: 15 , <O> = (-0.003538711470290323+0j)
RGsteps: 16 , <O> = (-0.014152758046130637+0j)
RGsteps: 17 , <O> = (-0.0564683107051954+0j)
RGsteps: 18 , <O> = (-0.21717823888768314+0j)
RGsteps: 19 , <O> = (-0.5621843217333166+0j)
RGsteps: 20 , <O> = (-0.6290102380000061+0j)
RGsteps: 21 , <O> = (-0.6290227074300213+0j)
RGsteps: 22 , <O> = (-0.6290227074300214+0j)


(-0.6290227074300214+5.745248156333163e-17j)

In [59]:
#exact value
mag_exact=(1-np.sinh(2/T)**(-4))**(1/8)
print(mag_exact)

0.629648162973007


scaling dimensions

In [225]:
i=3
A=A_list[i+1]
Aold,U,vl,vr,yl,yr,D,w,u,v=A_list[i],U_list[i],vl_list[i],vr_list[i],yl_list[i],yr_list[i],D_list[i],w_list[i],u_list[i],v_list[i]


gl=gl_(vl)
gr=gr_(vr)
gu=gu_(Aold,U,vl,vr)
gnw=gnw_(vl,vr,yr,D,w)
gne=gne_(vl,vr,yl,D,w)
gnw,gne=gauge_(gnw,gne,u,v)
chiwt=min(chiw,Aold.shape[1])
chiyt=min(chiy,Aold.shape[0])

def scaleop(v):
    M=v.reshape(chiwt,chiwt,chiyt,chiyt,chiyt,chiyt,chiwt,chiwt)
    Mscaled=rg_(M,gl,gr,gu,gnw,gne)
    return Mscaled.flatten()

n=chiw**4*chiy**4

In [233]:
import scipy
from scipy.sparse.linalg import LinearOperator

N_level=25
Atemp = LinearOperator((n,n), matvec = scaleop, dtype='complex')
ew = scipy.sparse.linalg.eigs(Atemp, k=N_level, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew=ew[::-1]
-np.log2(np.abs(ew/ew[0]))

array([-0.        ,  0.12677561,  1.00035411,  1.12314818,  1.12391534,
        2.00016536,  2.00021137,  2.0006836 ,  2.00107136,  2.1210812 ,
        2.12567347,  2.12778398,  2.99864148,  2.99916299,  2.99959015,
        3.00017952,  3.00198907,  3.10273781,  3.12477342,  3.12525128,
        3.12935573,  3.13098694,  3.13803997,  3.98877435,  3.99197728])

In [228]:
i=3
A=A_list[i+1]
Aold,U,vl,vr,yl,yr,D,w,u,v=A_list[i],U_list[i],vl_list[i],vr_list[i],yl_list[i],yr_list[i],D_list[i],w_list[i],u_list[i],v_list[i]


wgauged=w@u
# wgauged=w

def scaleop(v):
    M=v.reshape(chiw,chiw,chiw,chiw)
    mera=np.einsum(M,[0,1,2,3],U,[0,1,8,9],wgauged,[12,8,4],wgauged,[9,13,5],U.conj(),[2,3,10,11],wgauged.conj(),[12,10,6],wgauged.conj(),[11,13,7],optimize=("greedy",2**100))
    return mera.flatten()

n=chiw**4

import scipy
from scipy.sparse.linalg import LinearOperator

N_level=20
Atemp = LinearOperator((n,n), matvec = scaleop, dtype='complex')
ew = scipy.sparse.linalg.eigs(Atemp, k=20,which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew=ew[::-1]
-np.log2(np.abs(ew/ew[0]))

array([-0.        ,  0.18126637,  0.56473003,  0.7651075 ,  0.82930577,
        0.93717961,  1.15840441,  1.16340123,  1.21039609,  1.23688751,
        1.46377128,  1.54051149,  1.57518613,  1.57518613,  1.64407593,
        1.64407593,  1.64840365,  1.86336301,  1.8636821 ,  1.86368214])

In [11]:
# compress legs

# def wrenv_(A):
#     return np.einsum(A,[5,6,0,8],A,[4,7,1,6],A.conj(),[4,7,3,9],A.conj(),[5,9,2,8])
# def wlenv_(A):
#     return np.einsum(A,[0,6,5,8],A,[1,7,4,6],A.conj(),[3,7,4,9],A.conj(),[2,9,5,8])
# def wvenv_(A):
#     return np.einsum(A,[8,0,6,5],A,[6,1,7,4],A.conj(),[8,2,9,5],A.conj(),[9,3,7,4])
# chik=20
# wrenv=wrenv_(A)
# _,wr=compress(wrenv,chik)
# wlenv=wlenv_(A)
# _,wl=compress(wlenv,chik)
# wvenv=wvenv_(A)
# _,wv=compress(wvenv,chik)

# wrenv=wrenv_(Aold)
# _,wrold=compress(wrenv,chik)
# wlenv=wlenv_(Aold)
# _,wlold=compress(wlenv,chik)
# wvenv=wvenv_(Aold)
# _,wvold=compress(wvenv,chik)

# glw=np.einsum(gl,[3,4,1,2],wrold.conj(),[3,4,0])
# grw=np.einsum(gr,[3,4,1,2],wlold.conj(),[3,4,0])
# guw=np.einsum(gu,[3,4,1,2],wvold.conj(),[3,4,0])

# def scaleop(v):
#     v4=v.reshape(chik,chik,chik,chik)
#     out=np.einsum(v4,[8,9,10,11],glw,[10,19,21],grw,[8,18,20],guw,[9,22,23],guw.conj(),[11,16,17],gnw,[20,22,5,14],gne,[21,23,7,15],gnw.conj(),[18,16,4,12],gne.conj(),[19,17,6,13],wl,[4,5,0],wr,[6,7,2],wv,[14,15,1],wv.conj(),[12,13,3],optimize=('greedy',2**100))
#     return out.flatten()
# def rscaleop(v):
#     v4=v.reshape(chik,chik,chik,chik)
#     out=np.einsum(v4.conj(),[0,1,2,3],glw,[10,19,21],grw,[8,18,20],guw,[9,22,23],guw.conj(),[11,16,17],gnw,[20,22,5,14],gne,[21,23,7,15],gnw.conj(),[18,16,4,12],gne.conj(),[19,17,6,13],wl,[4,5,0],wr,[6,7,2],wv,[14,15,1],wv.conj(),[12,13,3],optimize=('greedy',2**100))
#     return out.flatten().conj()

# n=chik**4